In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
from transformers import TrainingArguments,DataCollatorWithPadding,Trainer,AutoTokenizer,AutoModelForSequenceClassification,DistilBertConfig ,AutoConfig,DistilBertForMaskedLM,DistilBert
from datasets import load_dataset
from google.colab import drive
from transformers import pipeline

import evaluate
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 35.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

In [ ]:
drive.mount('/content/drive/')
base_location_in_drive="/content/drive/MyDrive/Model extraction project/"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# VICTIM MODEL

In [ ]:
DistilBertForMaskedLM,
checkpoint="distilbert-base-uncased"
trained_news_checkpoint ='/content/drive/MyDrive/Model extraction project/victim_news_model/trained_news_model'
# use checkpoint to finetune the model and use the trained_checkpoint to get the model after finetuning.
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
config = AutoConfig.from_pretrained(checkpoint,id2label={0:'Sport',1:'Business'} ,num_labels = 2)
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
model = DistilBertForMaskedLM(config,id2label={0:'Sport',1:'Business'} ,num_labels = 2)

TypeError: ignored

In [ ]:
yahoo_dataset = load_dataset("yahoo_answers_topics")

Generating train split:   0%|          | 0/1400000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Dataset yahoo_answers_topics downloaded and prepared to /root/.cache/huggingface/datasets/yahoo_answers_topics/yahoo_answers_topics/1.0.0/0edb353eefe79d9245d7bd7cac5ae6af19530439da520d6dde1c206ee38f4439. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
yahoo_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer'],
        num_rows: 1400000
    })
    test: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer'],
        num_rows: 60000
    })
})

In [ ]:
#to change sport and business ids to 0 & 1
def change_label(x):
  new_labels=[0 if i==5 else 1 for i in x["topic"]]
  return {"label":new_labels}

In [ ]:

training_examples_length=1400000
label_5=label_6=count=0
# to leave only busines and sports each 3350 examples
def custom_filter(example):
  global label_5,label_6,count

  count+=1# number of times we called our fucntion= number of examples processed so far
  label=example['topic']

  if count<training_examples_length:# we are still in training examples


    if label==5:
      if label_5<3550:# 50%
        label_5+=1
        return True
      return False

    if label==6:
      if label_6<3550:# 50%
        label_6+=1
        return True
      return False
    else:
      return False

  else :# we are in test examples

    if label==5:
      if label_5<3550 + 887: # the extra 887 are 50% of the test
        label_5+=1
        return True
      return False

    if label==6:
      if label_6<3550 + 887: # the extra 887 are 50% of the test
        label_6+=1
        return True
      return False
    else:
      return False

In [ ]:
filtered_dataset =yahoo_dataset.filter(custom_filter)
filtered_dataset =filtered_dataset.map(change_label,batched=True)
filtered_dataset = filtered_dataset.shuffle(seed = 2)
filtered_dataset

Filter:   0%|          | 0/1400000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/60000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7100 [00:00<?, ? examples/s]

Map:   0%|          | 0/1774 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'label'],
        num_rows: 7100
    })
    test: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'label'],
        num_rows: 1774
    })
})

In [ ]:
def tokenizer_fuction(batch):
  return tokenizer(batch["best_answer"],truncation=True)

In [ ]:
#we add a validation split
dataset_news = filtered_dataset
dataset_news['validation']=dataset_news['test'].select(range(887))
dataset_news['test']=dataset_news['test'].select(range(887,len(dataset_news['test'])))
dataset_news

DatasetDict({
    train: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'label'],
        num_rows: 7100
    })
    test: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'label'],
        num_rows: 887
    })
    validation: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'label'],
        num_rows: 887
    })
})

In [ ]:
tokenized_dataset_news=dataset_news.map(tokenizer_fuction,batched=True)
tokenized_dataset_news

Map:   0%|          | 0/7100 [00:00<?, ? examples/s]

Map:   0%|          | 0/887 [00:00<?, ? examples/s]

Map:   0%|          | 0/887 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'label', 'input_ids', 'attention_mask'],
        num_rows: 7100
    })
    test: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'label', 'input_ids', 'attention_mask'],
        num_rows: 887
    })
    validation: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'label', 'input_ids', 'attention_mask'],
        num_rows: 887
    })
})

In [ ]:
tokenized_dataset_news_yahoo = tokenized_dataset_news

In [ ]:
cleaned_tokenized_dataset_news =tokenized_dataset_news.remove_columns(['id', 'topic', 'question_title', 'question_content'])
cleaned_tokenized_dataset_news

DatasetDict({
    train: Dataset({
        features: ['best_answer', 'label', 'input_ids', 'attention_mask'],
        num_rows: 7100
    })
    test: Dataset({
        features: ['best_answer', 'label', 'input_ids', 'attention_mask'],
        num_rows: 887
    })
    validation: Dataset({
        features: ['best_answer', 'label', 'input_ids', 'attention_mask'],
        num_rows: 887
    })
})

In [ ]:
def compute_metrics(logits_labels):
  logits,labels = logits_labels
  metric = evaluate.load('accuracy')
  preds = np.argmax(logits,axis = -1)
  return metric.compute(predictions = preds,references= labels)

In [ ]:
training_arguments=TrainingArguments(base_location_in_drive +"victim_news_model"
                                     ,num_train_epochs=4
                                     ,evaluation_strategy="steps"
                                     ,eval_steps=300
                                     ,save_steps=500
                                     ,overwrite_output_dir=True
                                     ,save_total_limit=1
                                     ,per_device_train_batch_size=16
                                     ,per_device_eval_batch_size= 16
                                     )

trainer=Trainer(model=model
                ,args=training_arguments
                ,tokenizer=tokenizer
                ,data_collator=data_collator
                ,compute_metrics=compute_metrics
                #,train_dataset=tokenized_dataset_news["train"]
                #,eval_dataset=tokenized_dataset_news["validation"]
                )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train('/content/drive/MyDrive/Model extraction project/victim_news_model/checkpoint-1500')
trainer.save_model("/content/drive/MyDrive/Model extraction project/victim_news_model/trained_news_model")

After Training 7100 examples for four epochs our accuracy was found to be **98.6** on training set,  **85.45** on validation set and **85.9** on test set

In [ ]:
pred_train=trainer.predict(tokenized_dataset_news["train"])
print(compute_metrics([pred_train.predictions,pred_train.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: topic, id, question_title, question_content, best_answer. If topic, id, question_title, question_content, best_answer are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 7100
  Batch size = 16


{'accuracy': 0.9866197183098592}


In [ ]:
pred_validation=trainer.predict(tokenized_dataset_news["validation"])
print(compute_metrics([pred_validation.predictions,pred_validation.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: best_answer, question_title, topic, id, question_content. If best_answer, question_title, topic, id, question_content are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 887
  Batch size = 16
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'accuracy': 0.85456595264938}


# Makind a pipeline for our victim model

In [ ]:
from transformers import pipeline
trained_news_checkpoint ='/content/drive/MyDrive/Model extraction project/victim_news_model/trained_news_model'
victim_news = pipeline('sentiment-analysis',model =trained_news_checkpoint)

In [ ]:
victim_news('i love Chelsea')

[{'label': 'Sport', 'score': 0.9996089339256287}]

In [ ]:
victim_news('i visited wall street')

[{'label': 'Business', 'score': 0.9700660109519958}]

# Finally: We find accuracy of our model on its test set

In [ ]:
pred_test=trainer.predict(tokenized_dataset_news_yahoo["test"])
print(compute_metrics([pred_test.predictions,pred_test.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: best_answer, id, question_content, topic, question_title. If best_answer, id, question_content, topic, question_title are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 887
  Batch size = 16


{'accuracy': 0.859075535512965}


# The accuracy of the victim model was found to be **(85.9)** on test set

# **EXTRACTION (1)**

# Preparing dataset for extraction

In [ ]:
exlabels_train = np.argmax(pred_train.predictions,axis = -1)
exlabels_validation = np.argmax(pred_validation.predictions,axis = -1)
exlabels_test = np.argmax(pred_test.predictions,axis = -1)

In [ ]:
exlabels = np.concatenate((exlabels_train , exlabels_test ,exlabels_validation ))
len(exlabels)

8874

In [ ]:
#num_training = len(exlabels)
count = -1
def ex_labels(example):
  global count
  count+=1
  return{'ex_labels':exlabels[count] }

In [ ]:
tokenized_dataset_news = tokenized_dataset_news.map(ex_labels)
tokenized_dataset_news

Map:   0%|          | 0/7100 [00:00<?, ? examples/s]

Map:   0%|          | 0/887 [00:00<?, ? examples/s]

Map:   0%|          | 0/887 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'label', 'input_ids', 'attention_mask', 'ex_labels'],
        num_rows: 7100
    })
    test: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'label', 'input_ids', 'attention_mask', 'ex_labels'],
        num_rows: 887
    })
    validation: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'label', 'input_ids', 'attention_mask', 'ex_labels'],
        num_rows: 887
    })
})

In [ ]:
extacted_tokenized_dataset_news = tokenized_dataset_news.remove_columns(['id', 'topic', 'question_title', 'question_content', 'label'])
extacted_tokenized_dataset_news = extacted_tokenized_dataset_news.rename_column('ex_labels','label')
extacted_tokenized_dataset_news

DatasetDict({
    train: Dataset({
        features: ['best_answer', 'input_ids', 'attention_mask', 'label'],
        num_rows: 7100
    })
    test: Dataset({
        features: ['best_answer', 'input_ids', 'attention_mask', 'label'],
        num_rows: 887
    })
    validation: Dataset({
        features: ['best_answer', 'input_ids', 'attention_mask', 'label'],
        num_rows: 887
    })
})

In [ ]:
#debugging:
# count = 0
# for i in range(len(tokenized_dataset_news['validation']['label'])):
#   if(tokenized_dataset_news['validation']['label'][i] != extacted_tokenized_dataset_news['test']['label'][i]):
#     count+=1
# count

using the same dataset used for training victim model

In [ ]:
checkpoint="distilbert-base-uncased"
trained_news_checkpoint ='/content/drive/MyDrive/Model extraction project/victim_news_model/trained_news_model'
extracted_news1_checkpoint ='/content/drive/MyDrive/Model extraction project/extracted_news_model/extracted_news_model_one'
# use checkpoint to finetune the model and use the trained_checkpoint to get the model after finetuning.
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(extracted_news1_checkpoint,id2label={0:'Sport',1:'Business'})

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapsh

In [ ]:
def compute_metrics(logits_labels):
  logits,labels = logits_labels
  metric = evaluate.load('accuracy')
  preds = np.argmax(logits,axis = -1)
  return metric.compute(predictions = preds,references= labels)

In [ ]:
training_arguments=TrainingArguments(base_location_in_drive +"extracted_news_model"
                                     ,num_train_epochs=4
                                     ,evaluation_strategy="steps"
                                     ,eval_steps=300
                                     ,logging_steps=300
                                     ,save_steps=1000
                                     ,overwrite_output_dir=True
                                     ,save_total_limit=1
                                     ,per_device_train_batch_size=16
                                     ,per_device_eval_batch_size= 16
                                     )

trainer_one=Trainer(model=model
                ,args=training_arguments
                ,tokenizer=tokenizer
                ,data_collator=data_collator
                ,compute_metrics=compute_metrics
                ,train_dataset=extacted_tokenized_dataset_news["train"]
                ,eval_dataset=extacted_tokenized_dataset_news["validation"]
                )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer_one.train()
trainer_one.save_model("/content/drive/MyDrive/Model extraction project/extracted_news_model/extracted_news_model_one")

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: best_answer. If best_answer are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 7100
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1776
  Number of trainable parameters = 66955010


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss,Accuracy
300,0.067600,0.234060,0.936866
600,0.055800,0.392272,0.935738
900,0.036600,0.291714,0.934611
1200,0.007100,0.339304,0.945885
1500,0.005300,0.382287,0.944758


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: best_answer. If best_answer are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 887
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: best_answer. If best_answer are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 887
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: best_answer. If best_answer are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Ru

we estimate accuracy of our extracted model with reference to the labels extracted from victim model

In [ ]:
pred_train=trainer_one.predict(extacted_tokenized_dataset_news["train"])
print(compute_metrics([pred_train.predictions,pred_train.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: best_answer. If best_answer are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 7100
  Batch size = 16


{'accuracy': 0.9998591549295774}


In [ ]:
pred_validation=trainer_one.predict(extacted_tokenized_dataset_news["validation"])
print(compute_metrics([pred_validation.predictions,pred_validation.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: best_answer. If best_answer are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 887
  Batch size = 16


{'accuracy': 0.943630214205186}


In [ ]:
pred_test=trainer_one.predict(extacted_tokenized_dataset_news["test"])
print(compute_metrics([pred_test.predictions,pred_test.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: best_answer. If best_answer are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 887
  Batch size = 16


{'accuracy': 0.939120631341601}


we estimate accuracy of our extracted model with reference to the labels extracted from original dataset

In [ ]:
pred_train=trainer_one.predict(tokenized_dataset_news["train"])
print(compute_metrics([pred_train.predictions,pred_train.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: topic, ex_labels, id, question_title, question_content, best_answer. If topic, ex_labels, id, question_title, question_content, best_answer are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 7100
  Batch size = 16


{'accuracy': 0.9864788732394366}


In [ ]:
pred_validation=trainer_one.predict(tokenized_dataset_news["validation"])
print(compute_metrics([pred_validation.predictions,pred_validation.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: topic, ex_labels, id, question_title, question_content, best_answer. If topic, ex_labels, id, question_title, question_content, best_answer are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 887
  Batch size = 16


{'accuracy': 0.85456595264938}


# We finally compute accuracy of our model compared to test set of the victim model:

In [ ]:
pred_test=trainer_one.predict(tokenized_dataset_news_yahoo["test"])
print(compute_metrics([pred_test.predictions,pred_test.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: best_answer, id, question_content, topic, question_title. If best_answer, id, question_content, topic, question_title are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 887
  Batch size = 16
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'accuracy': 0.8703494926719278}


#We found that the accuracy of the extracted model equals 87.034 on the test set of the victim model

# Making a pipeline for our first extracted model (Da = Dv)

In [ ]:
from transformers import pipeline
extracted_news1_checkpoint ='/content/drive/MyDrive/Model extraction project/extracted_news_model/extracted_news_model_one'
extracted_news1 = pipeline('sentiment-analysis',model =extracted_news1_checkpoint)

In [ ]:
extracted_news1('utilitarianism')

[{'label': 'Business', 'score': 0.9999470710754395}]

In [ ]:
extracted_news1('Get Potter out of my house!')

[{'label': 'Business', 'score': 0.9999699592590332}]

In [ ]:
extracted_news1('i visited wall street')

[{'label': 'Business', 'score': 0.9999237060546875}]

# EXTRACTION (2)
Using ag_news dataset as queries for extracting victim model                                                                        
The dataset used for extraction here is equal to the dataset used for training victim model in term of size

In [ ]:
from datasets import load_dataset
ag_dataset = load_dataset("ag_news")

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
ag_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [ ]:
#to change sport and business ids to 0 & 1
def change_label_ag(x):
  new_labels=[0 if i==1 else 1 for i in x["label"]]
  return {"label":new_labels}

In [ ]:

training_examples_length=120000
label_1=label_2=count=0
# to leave only busines and sports each 3350 examples
def custom_filter_ag(example):
  global label_1,label_2,count

  count+=1# number of times we called our fucntion= number of examples processed so far
  label=example['label']

  if count<=training_examples_length:# we are still in training examples

    if label==1:
      if label_1<3550:# 50%
        label_1+=1
        return True
      return False

    if label==2:
      if label_2<3550:# 50%
        label_2+=1
        return True
      return False
    else:
      return False

  else :# we are in test examples

    if label==1:
      if label_1<3550 + 887: # the extra 887 are 50% of the test
        label_1+=1
        return True
      return False

    if label==2:
      if label_2<3550 + 887: # the extra 887 are 50% of the test
        label_2+=1
        return True
      return False
    else:
      return False

In [ ]:
filtered_dataset = ag_dataset.filter(custom_filter_ag)
filtered_dataset = filtered_dataset.map(change_label_ag,batched=True)
filtered_dataset = filtered_dataset.shuffle(seed = 2)
filtered_dataset

Filter:   0%|          | 0/120000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/7100 [00:00<?, ? examples/s]

Map:   0%|          | 0/1774 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1774
    })
})

In [ ]:
def tokenizer_fuction(batch):
  return tokenizer(batch["text"],truncation=True)

In [ ]:
#we add a validation split
dataset_news = filtered_dataset
dataset_news['validation']=dataset_news['test'].select(range(887))
dataset_news['test']=dataset_news['test'].select(range(887,len(dataset_news['test'])))
dataset_news

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 887
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 887
    })
})

In [ ]:
checkpoint="distilbert-base-uncased"
trained_news_checkpoint ='/content/drive/MyDrive/Model extraction project/victim_news_model/trained_news_model'
extracted_news_ag1_checkpoint ='/content/drive/MyDrive/Model extraction project/extracted_news_model/exracted_news_model_ag1'
# use checkpoint to finetune the model and use the trained_checkpoint to get the model after finetuning.
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(extracted_news_ag1_checkpoint,id2label={0:'Sport',1:'Business'})
victim_model = AutoModelForSequenceClassification.from_pretrained(trained_news_checkpoint,id2label={0:'Sport',1:'Business'})

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapsh

In [ ]:
tokenized_dataset_news=dataset_news.map(tokenizer_fuction,batched=True)
tokenized_dataset_news

In [ ]:
def compute_metrics(logits_labels):
  logits,labels = logits_labels
  metric = evaluate.load('accuracy')
  preds = np.argmax(logits,axis = -1)
  return metric.compute(predictions = preds,references= labels)

In [ ]:
training_arguments=TrainingArguments(base_location_in_drive +"victim_model"
                                     ,num_train_epochs=4
                                     ,evaluation_strategy="steps"
                                     ,eval_steps=300
                                     ,logging_steps=300
                                     ,save_steps=1000
                                     ,overwrite_output_dir=True
                                     ,save_total_limit=1
                                     ,per_device_train_batch_size=16
                                     ,per_device_eval_batch_size= 16
                                     )

trainer_victim=Trainer(model=victim_model
                ,args=training_arguments
                ,tokenizer=tokenizer
                ,data_collator=data_collator
                ,compute_metrics=compute_metrics
                #,train_dataset=tokenized_dataset_news["train"]
                #,eval_dataset=tokenized_dataset_news["validation"]
                )

In [ ]:
pred_train=trainer_victim.predict(tokenized_dataset_news["train"])
print(compute_metrics([pred_train.predictions,pred_train.label_ids]))

In [ ]:
pred_validation=trainer_victim.predict(tokenized_dataset_news["validation"])
print(compute_metrics([pred_validation.predictions,pred_validation.label_ids]))

In [ ]:
pred_test=trainer_victim.predict(tokenized_dataset_news["test"])
print(compute_metrics([pred_test.predictions,pred_test.label_ids]))

we found that our victim model generate queries with  generate queries with accuracy of 96.7 on training set and 96.84 on validation set and 97.4 on test set     

we use these generated queries for extraction

# ready for extraction

In [ ]:
exlabels_train = np.argmax(pred_train.predictions,axis = -1)
exlabels_validation = np.argmax(pred_validation.predictions,axis = -1)
exlabels_test = np.argmax(pred_test.predictions,axis = -1)

In [ ]:
exlabels = np.concatenate((exlabels_train , exlabels_test ,exlabels_validation ))
len(exlabels)

39187

In [ ]:
#num_training = len(exlabels)
count = -1
def ex_labels(example):
  global count
  count+=1
  return{'ex_labels':exlabels[count] }

In [ ]:
tokenized_dataset_news = tokenized_dataset_news.map(ex_labels)
tokenized_dataset_news

Map:   0%|          | 0/7100 [00:00<?, ? examples/s]

Map:   0%|          | 0/887 [00:00<?, ? examples/s]

Map:   0%|          | 0/887 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'label', 'input_ids', 'attention_mask', 'ex_labels'],
        num_rows: 7100
    })
    test: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'label', 'input_ids', 'attention_mask', 'ex_labels'],
        num_rows: 887
    })
    validation: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'label', 'input_ids', 'attention_mask', 'ex_labels'],
        num_rows: 887
    })
})

In [ ]:
extacted_tokenized_dataset_news = tokenized_dataset_news.remove_columns(['label'])
extacted_tokenized_dataset_news = extacted_tokenized_dataset_news.rename_column('ex_labels','label')
extacted_tokenized_dataset_news

DatasetDict({
    train: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'input_ids', 'attention_mask', 'label'],
        num_rows: 7100
    })
    test: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'input_ids', 'attention_mask', 'label'],
        num_rows: 887
    })
    validation: Dataset({
        features: ['id', 'topic', 'question_title', 'question_content', 'best_answer', 'input_ids', 'attention_mask', 'label'],
        num_rows: 887
    })
})

In [ ]:
#debugging:
count = 0
for i in range(len(tokenized_dataset_news['validation']['label'])):
  if(tokenized_dataset_news['train']['label'][i] != extacted_tokenized_dataset_news['train']['label'][i]):
    count+=1
count

KeyboardInterrupt: ignored

In [ ]:
def compute_metrics(logits_labels):
  logits,labels = logits_labels
  metric = evaluate.load('accuracy')
  preds = np.argmax(logits,axis = -1)
  return metric.compute(predictions = preds,references= labels)

In [ ]:
training_arguments=TrainingArguments(base_location_in_drive +"extracted_news_model"
                                     ,num_train_epochs=4
                                     ,evaluation_strategy="steps"
                                     ,eval_steps=300
                                     ,logging_steps=300
                                     ,save_steps=1000
                                     ,overwrite_output_dir=True
                                     ,save_total_limit=1
                                     ,per_device_train_batch_size=16
                                     ,per_device_eval_batch_size= 16
                                     )

trainer_two=Trainer(model=model
                ,args=training_arguments
                ,tokenizer=tokenizer
                ,data_collator=data_collator
                ,compute_metrics=compute_metrics
                #,train_dataset=extacted_tokenized_dataset_news["train"]
                #,eval_dataset=extacted_tokenized_dataset_news["validation"]
                )

In [ ]:
trainer_two.train()
trainer_two.save_model("/content/drive/MyDrive/Model extraction project/extracted_news_model/exracted_news_model_ag1")

we estimate accuracy of our extracted model with reference to the labels extracted from victim model

In [ ]:
pred_train=trainer_two.predict(extacted_tokenized_dataset_news["train"])
print(compute_metrics([pred_train.predictions,pred_train.label_ids]))

In [ ]:
pred_validation=trainer_two.predict(extacted_tokenized_dataset_news["validation"])
print(compute_metrics([pred_validation.predictions,pred_validation.label_ids]))

In [ ]:
pred_test=trainer_two.predict(extacted_tokenized_dataset_news["test"])
print(compute_metrics([pred_test.predictions,pred_test.label_ids]))

we estimate accuracy of our extracted model with reference to the labels extracted from the original ag_news dataset

In [ ]:
pred_train=trainer_two.predict(tokenized_dataset_news["train"])
print(compute_metrics([pred_train.predictions,pred_train.label_ids]))

In [ ]:
pred_validation=trainer_two.predict(tokenized_dataset_news["validation"])
print(compute_metrics([pred_validation.predictions,pred_validation.label_ids]))

In [ ]:
pred_test=trainer_two.predict(tokenized_dataset_news["test"])
print(compute_metrics([pred_test.predictions,pred_test.label_ids]))

# We finally compute accuracy of our model compared to test set of the victim model:

In [ ]:
pred_test=trainer_two.predict(tokenized_dataset_news_yahoo["test"])
print(compute_metrics([pred_test.predictions,pred_test.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: best_answer, id, question_content, topic, question_title. If best_answer, id, question_content, topic, question_title are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 887
  Batch size = 16
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'accuracy': 0.8105975197294251}


# We found that the accuracy of the extracted model equals 81.059 on the test set of the victim model

# Making a pipeline for our second extracted model                                (Da ≠ Dv) (1x)

In [ ]:
from transformers import pipeline
extracted_news_ag1_checkpoint ='/content/drive/MyDrive/Model extraction project/extracted_news_model/exracted_news_model_ag1'
extracted_news_ag1 = pipeline('sentiment-analysis',model =extracted_news_ag1_checkpoint)

In [ ]:
extracted_news_ag1('i love Al-Hilal')

[{'label': 'Sport', 'score': 0.9948082566261292}]

In [ ]:
extracted_news1('i visited wall street')

[{'label': 'Business', 'score': 0.9999237060546875}]

#**EXTRACTION (3)**


Using ag_news dataset as queries for extracting victim model                                                                        
The dataset used for extraction here equals five times the dataset used for training victim model in term of size

In [ ]:
from datasets import load_dataset
ag_dataset = load_dataset("ag_news")

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
ag_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})

In [ ]:
#to change sport and business ids to 0 & 1
def change_label_ag(x):
  new_labels=[0 if i==1 else 1 for i in x["label"]]
  return {"label":new_labels}

In [ ]:

training_examples_length=120000
label_1=label_2=count=0
# to leave only busines and sports each 3350 examples
def custom_filter_ag5(example):
  global label_1,label_2,count

  count+=1# number of times we called our fucntion= number of examples processed so far
  label=example['label']

  if count<=training_examples_length:# we are still in training examples

    if label==1:
      if label_1<(17750+1400):# 50% (3550*5)(1900 to be used for validation later)
        label_1+=1
        return True
      return False

    if label==2:
      if label_2<(17750+1400):# 50%  3550*5)(1900 to be used for validation later)
        label_2+=1
        return True
      return False
    else:
      return False

  else :# we are in test examples

    if label==1:
      if label_1< (17750+1400 + 1900): # the extra 1900 are 50% of the test
        label_1+=1
        return True
      return False

    if label==2:
      if label_2< (17750+1400 + 1900): # the extra 1900 are 50% of the test
        label_2+=1
        return True
      return False
    else:
      return False

In [ ]:
filtered_dataset = ag_dataset.filter(custom_filter_ag5)
filtered_dataset = filtered_dataset.map(change_label_ag,batched=True)
filtered_dataset = filtered_dataset.shuffle(seed = 2)
filtered_dataset

Filter:   0%|          | 0/7600 [00:00<?, ? examples/s]

Map:   0%|          | 0/3800 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 38300
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3800
    })
})

In [ ]:
def tokenizer_fuction(batch):
  return tokenizer(batch["text"],truncation=True)

In [ ]:
#we add a validation split
dataset_news = filtered_dataset
dataset_news['validation']=dataset_news['train'].select(range(2800))
dataset_news['train']=dataset_news['train'].select(range(2800,len(dataset_news['train'])))
dataset_news

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 35500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3800
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2800
    })
})

In [ ]:
checkpoint="distilbert-base-uncased"
trained_news_checkpoint ='/content/drive/MyDrive/Model extraction project/victim_news_model/trained_news_model'
extracted_news_ag5_checkpoint = '/content/drive/MyDrive/Model extraction project/extracted_news_model/exracted_news_model_ag5'
# use checkpoint to finetune the model and use the trained_checkpoint to get the model after finetuning.
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(extracted_news_ag5_checkpoint,id2label={0:'Sport',1:'Business'})
victim_model = AutoModelForSequenceClassification.from_pretrained(trained_news_checkpoint,id2label={0:'Sport',1:'Business'})

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.26.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapsh

In [ ]:
tokenized_dataset_news=dataset_news.map(tokenizer_fuction,batched=True)
tokenized_dataset_news

In [ ]:
def compute_metrics(logits_labels):
  logits,labels = logits_labels
  metric = evaluate.load('accuracy')
  preds = np.argmax(logits,axis = -1)
  return metric.compute(predictions = preds,references= labels)

In [ ]:
training_arguments=TrainingArguments(base_location_in_drive +"extracted_news_model"
                                     ,num_train_epochs=4
                                     ,evaluation_strategy="steps"
                                     ,eval_steps=300
                                     ,logging_steps=300
                                     ,save_steps=1000
                                     ,overwrite_output_dir=True
                                     ,save_total_limit=1
                                     ,per_device_train_batch_size=16
                                     ,per_device_eval_batch_size= 16
                                     )

trainer_victim=Trainer(model=victim_model
                ,args=training_arguments
                ,tokenizer=tokenizer
                ,data_collator=data_collator
                ,compute_metrics=compute_metrics
                #,train_dataset=tokenized_dataset_news["train"]
                #,eval_dataset=tokenized_dataset_news["validation"]
                )

In [ ]:
pred_train=trainer_victim.predict(tokenized_dataset_news["train"])
print(compute_metrics([pred_train.predictions,pred_train.label_ids]))

In [ ]:
pred_validation=trainer_victim.predict(tokenized_dataset_news["validation"])
print(compute_metrics([pred_validation.predictions,pred_validation.label_ids]))

In [ ]:
pred_test=trainer_victim.predict(tokenized_dataset_news["test"])
print(compute_metrics([pred_test.predictions,pred_test.label_ids]))

we found that our victim model generate queries with accuracy of 96.97 on training set and 97.21 on validation set and 96.6 on test set     

we use these generated queries for extraction

# ready for extraction

In [ ]:
exlabels_train = np.argmax(pred_train.predictions,axis = -1)
exlabels_validation = np.argmax(pred_validation.predictions,axis = -1)
exlabels_test = np.argmax(pred_test.predictions,axis = -1)

In [ ]:
exlabels = np.concatenate((exlabels_train , exlabels_test ,exlabels_validation ))
len(exlabels)

42100

In [ ]:
#num_training = len(exlabels)
count = -1
def ex_labels(example):
  global count
  count+=1
  return{'ex_labels':exlabels[count] }

In [ ]:
tokenized_dataset_news = tokenized_dataset_news.map(ex_labels)
tokenized_dataset_news

Map:   0%|          | 0/35500 [00:00<?, ? examples/s]

Map:   0%|          | 0/3800 [00:00<?, ? examples/s]

Map:   0%|          | 0/2800 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'ex_labels'],
        num_rows: 35500
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'ex_labels'],
        num_rows: 3800
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask', 'ex_labels'],
        num_rows: 2800
    })
})

In [ ]:
extacted_tokenized_dataset_news = tokenized_dataset_news.remove_columns(['label'])
extacted_tokenized_dataset_news = extacted_tokenized_dataset_news.rename_column('ex_labels','label')
extacted_tokenized_dataset_news

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'label'],
        num_rows: 35500
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'label'],
        num_rows: 3800
    })
    validation: Dataset({
        features: ['text', 'input_ids', 'attention_mask', 'label'],
        num_rows: 2800
    })
})

In [ ]:
# #debugging:
# count = 0
# for i in range(len(tokenized_dataset_news['validation']['label'])):
#   if(tokenized_dataset_news['validation']['label'][i] != extacted_tokenized_dataset_news['validation']['label'][i]):
#     count+=1
# count

78

In [ ]:
def compute_metrics(logits_labels):
  logits,labels = logits_labels
  metric = evaluate.load('accuracy')
  preds = np.argmax(logits,axis = -1)
  return metric.compute(predictions = preds,references= labels)

In [ ]:
training_arguments=TrainingArguments(base_location_in_drive +"extracted_news_model"
                                     ,num_train_epochs=4
                                     ,evaluation_strategy="steps"
                                     ,eval_steps=300
                                     ,logging_steps=300
                                     ,save_steps=1000
                                     ,overwrite_output_dir=True
                                     ,save_total_limit=1
                                     ,per_device_train_batch_size=16
                                     ,per_device_eval_batch_size= 16
                                     )

trainer_three=Trainer(model=model
                ,args=training_arguments
                ,tokenizer=tokenizer
                ,data_collator=data_collator
                ,compute_metrics=compute_metrics
                ,train_dataset=extacted_tokenized_dataset_news["train"]
                ,eval_dataset=extacted_tokenized_dataset_news["validation"]
                )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer_three.train()
trainer_three.save_model("/content/drive/MyDrive/Model extraction project/extracted_news_model/exracted_news_model_ag5")

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 35500
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 8876
  Number of trainable parameters = 66955010


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss,Accuracy
300,0.120200,0.082520,0.976786
600,0.087300,0.069993,0.976071
900,0.085900,0.097268,0.977143
1200,0.077200,0.065999,0.982857
1500,0.074200,0.080379,0.980000
1800,0.074500,0.063415,0.980357
2100,0.070600,0.065649,0.978929
2400,0.045400,0.085545,0.982143
2700,0.030900,0.083615,0.981071
3000,0.023900,0.090352,0.980000


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2800
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2800
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 

we estimate accuracy of our extracted model with reference to the labels extracted from victim model

In [ ]:
pred_train=trainer_three.predict(extacted_tokenized_dataset_news["train"])
print(compute_metrics([pred_train.predictions,pred_train.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 35500
  Batch size = 16


{'accuracy': 0.9998309859154929}


In [ ]:
pred_validation=trainer_three.predict(extacted_tokenized_dataset_news["validation"])
print(compute_metrics([pred_validation.predictions,pred_validation.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2800
  Batch size = 16


{'accuracy': 0.9839285714285714}


In [ ]:
pred_test=trainer_three.predict(extacted_tokenized_dataset_news["test"])
print(compute_metrics([pred_test.predictions,pred_test.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3800
  Batch size = 16


{'accuracy': 0.9818421052631578}


we estimate accuracy of our extracted model with reference to the labels extracted from the original ag_news dataset

In [ ]:
pred_train=trainer_three.predict(tokenized_dataset_news["train"])
print(compute_metrics([pred_train.predictions,pred_train.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: ex_labels, text. If ex_labels, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 35500
  Batch size = 16


{'accuracy': 0.969887323943662}


In [ ]:
pred_validation=trainer_three.predict(tokenized_dataset_news["validation"])
print(compute_metrics([pred_validation.predictions,pred_validation.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: ex_labels, text. If ex_labels, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 2800
  Batch size = 16


{'accuracy': 0.9775}


In [ ]:
pred_test=trainer_three.predict(tokenized_dataset_news["test"])
print(compute_metrics([pred_test.predictions,pred_test.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: ex_labels, text. If ex_labels, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3800
  Batch size = 16


{'accuracy': 0.9778947368421053}


# We finally compute accuracy of our model compared to test set of the victim model:

In [ ]:
pred_test=trainer_three.predict(tokenized_dataset_news_yahoo["test"])
print(compute_metrics([pred_test.predictions,pred_test.label_ids]))

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: best_answer, id, question_content, topic, question_title. If best_answer, id, question_content, topic, question_title are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 887
  Batch size = 16
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'accuracy': 0.8072153325817362}


# We found that the accuracy of the extracted model equals 80.72 on the test set of the victim model

# Making a pipeline for our third extracted model  (Da ≠ Dv) (5x)

In [ ]:
from transformers import pipeline
extracted_news_ag5_checkpoint = '/content/drive/MyDrive/Model extraction project/extracted_news_model/exracted_news_model_ag5'
extracted_news_ag5 = pipeline('sentiment-analysis',model =extracted_news_ag5_checkpoint)

In [ ]:
extracted_news_ag5('i love swimming')

[{'label': 'Sport', 'score': 0.9997968077659607}]

In [ ]:
extracted_news_ag5('i visited wall street')

[{'label': 'Business', 'score': 0.999853253364563}]